In [ ]:
from math import sin
import numpy as np
import sympy as sp
from .dynamic_model import DynamicModelBase
from utils.Logger import logger
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib as mpl
import math
from sympy.utilities.lambdify import lambdify
from scipy.linalg import expm, logm

class ODAR:
    def __init__(self):
        self.m = 1.72
        self.J = np.array([[0.5*0.27, 0.007*0.27, -0.001*0.27],
                           [0.007*0.27, 1.609*0.27, 0],
                           [-0.001*0.27, 0, 1.595*0.27]])
        self.L = 1.0
        self.g = 9.81
        self.dt = 0.01
        self.time_end = 10
        self.position = np.array([0, 0, 0], dtype=np.float64)
        self.velocity = np.array([0, 0, 0], dtype=np.float64)
        self.orientation = np.eye(3)
        self.angular_velocity = np.array([0, 0, 0], dtype=np.float64)
        self.B_ft = np.array([])

    def rotation_matrix(self, angles):
        return expm(np.cross(np.eye(3), angles))

    def play(self):
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')

        trajectory = [self.position.copy()]

        for _ in np.arange(0, self.time_end, self.dt):
            control_input = self.compute_control_input()
            state = self.evaluate_dynamics(self.get_state(), control_input)
            self.set_state(state)
            trajectory.append(self.position.copy())

        trajectory = np.array(trajectory)
        ax.plot3D(trajectory[:, 0], trajectory[:, 1], trajectory[:, 2], 'gray')
        plt.show()

    def compute_control_input(self):
        return np.zeros(4)

    def get_state(self):
        return np.hstack((self.position, self.velocity, self.orientation.flatten(), self.angular_velocity))

    def set_state(self, state):
        self.position, self.velocity, self.orientation, self.angular_velocity = np.split(state, [3, 6, 15])

    def evaluate_dynamics(self, current_state, control_input):
        x_u = np.hstack((current_state, control_input))
        next_state = self.dynamic_function(x_u)
        return next_state
